In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

os.chdir("..")

In [ ]:
import glob
import pickle
import time
from copy import deepcopy

import numpy as np

from retarget.ik_solver import IKSolver
from retarget.robot import Robot
from retarget.utils.configs import load_config
from retarget.visualizer import RobotVisualizer

In [ ]:
config = load_config("configs/gr1_teleop.yaml")

In [ ]:
robot = Robot(config)
visualzier = RobotVisualizer(robot, config)

In [ ]:
input_paths = glob.glob("data/*")
input_paths.sort()
input_path = input_paths[-1]

with open(input_path, "rb") as f:
    data = np.load(f, allow_pickle=True)

In [ ]:
left_pose_visualize_params = {
    "origin_color": 0xFF0000,
    "axis_length": 0.1,
    "axis_thickness": 0.01,
    "origin_radius": 0.03,
}

right_pose_visualize_params = {
    "origin_color": 0x00FF00,
    "axis_length": 0.1,
    "axis_thickness": 0.01,
    "origin_radius": 0.03,
}

head_pose_visualize_params = {
    "origin_color": 0x0000FF,
    "axis_length": 0.1,
    "axis_thickness": 0.01,
    "origin_radius": 0.03,
}

In [ ]:
head_axis_transform = np.array(
    [0.0, -1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
).reshape(4, 4)

left_wrist_axis_transform = np.array(
    [0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
).reshape(4, 4)

right_wrist_axis_transform = np.array(
    [0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
).reshape(4, 4)


l_elbow_pitch_idx = robot.joint2idx["l_elbow_pitch"]
r_elbow_pitch_idx = robot.joint2idx["r_elbow_pitch"]

q = deepcopy(robot.configuration.q)
q[l_elbow_pitch_idx] = np.pi / 2
q[r_elbow_pitch_idx] = -np.pi / 2

robot.update(q)
robot_head_pose, robot_l_wrist_pose, robot_r_wrist_pose = robot.get_link_transformations(
    ["link_head_roll", "link_LArm7", "link_RArm7"]
)


head_pose = data[0]["head"] @ head_axis_transform
relative_transfrom = robot_head_pose @ np.linalg.inv(head_pose)
# relative_transfrom[:3, 3] = 0.0

data_new = []
for data_t in data:
    left_wrist_pose = relative_transfrom @ (data_t["left_wrist"] @ left_wrist_axis_transform)
    right_wrist_pose = relative_transfrom @ (data_t["right_wrist"] @ right_wrist_axis_transform)
    head_pose = relative_transfrom @ (data_t["head"] @ head_axis_transform)
    data_new.append(
        {"left_wrist": left_wrist_pose, "right_wrist": right_wrist_pose, "head": head_pose}
    )


left_wrist_delta = deepcopy(data_new[0]["left_wrist"][0, :3, 3] - robot_l_wrist_pose[:3, 3])
right_wrist_delta = deepcopy(data_new[0]["right_wrist"][0, :3, 3] - robot_r_wrist_pose[:3, 3])

# relative_transfrom = np.eye(4)

In [ ]:
for data_t in data_new:
    left_wrist_pose = deepcopy(data_t["left_wrist"])
    left_wrist_pose[0, :3, 3] -= left_wrist_delta
    right_wrist_pose = deepcopy(data_t["right_wrist"])
    right_wrist_pose[0, :3, 3] -= right_wrist_delta
    head_pose = data_t["head"]
    visualzier.visualize(
        robot_state=robot.configuration.q,
        pose_dict={
            "left_wrist": (left_wrist_pose, left_pose_visualize_params),
            "right_wrist": (right_wrist_pose, right_pose_visualize_params),
            "head": (head_pose, head_pose_visualize_params),
            "robot_left_wrist": (robot_l_wrist_pose, left_pose_visualize_params),
            "robot_right_wrist": (robot_r_wrist_pose, right_pose_visualize_params),
            "robot_head": (robot_head_pose, head_pose_visualize_params),
        },
    )
    # break
    time.sleep(0.05)

In [ ]:
solver = IKSolver(robot, config)

In [ ]:
for data_t in data_new:
    left_wrist_pose = deepcopy(data_t["left_wrist"])
    left_wrist_pose[0, :3, 3] -= left_wrist_delta
    right_wrist_pose = deepcopy(data_t["right_wrist"])
    right_wrist_pose[0, :3, 3] -= right_wrist_delta
    _ = solver.solve({"link_LArm7": left_wrist_pose[0], "link_RArm7": right_wrist_pose[0]})

    robot_head_pose, robot_l_wrist_pose, robot_r_wrist_pose = robot.get_link_transformations(
        ["link_head_roll", "link_LArm7", "link_RArm7"]
    )
    head_pose = data_t["head"]
    visualzier.visualize(
        robot_state=robot.configuration.q,
        pose_dict={
            "left_wrist": (left_wrist_pose, left_pose_visualize_params),
            "right_wrist": (right_wrist_pose, right_pose_visualize_params),
            "head": (head_pose, head_pose_visualize_params),
            "robot_left_wrist": (robot_l_wrist_pose, left_pose_visualize_params),
            "robot_right_wrist": (robot_r_wrist_pose, right_pose_visualize_params),
            "robot_head": (robot_head_pose, head_pose_visualize_params),
        },
    )
    # break
    time.sleep(0.05)